In [100]:
import pandas as pd
import geopandas as gpd
import requests
from datetime import date
import plotly.express as px
import numpy as np
import time as time
from shapely.geometry import Polygon
import json

In [2]:
# this is importing a local config file with api keys.
# if you do not have access to the author's api keys, you should obtain your own and replace in your environment
import config
ipc_api_key = "&key="+config.ipc_key

In [8]:
url_base = "https://api.ipcinfo.org/"

def get_ipc_data(endpoint, format):
    time.sleep(2)
    request = requests.get(url=url_base+endpoint+ipc_api_key)
    print(request)
    #  print ("length is " + str(len(request.json()["dataset"])))
    if format == "pd":
        return pd.DataFrame(request.json())
    elif format == "json":
        return request.json()
    elif format == "gpd":
        return gpd.GeoDataFrame(request.json())
    else:
        return request.text
    

In [60]:
def make_grid(gdf, size):
    xmin, ymin, xmax, ymax = gdf.total_bounds

    length = size
    wide = size

    cols = list(np.arange(xmin, xmax + size, size))
    rows = list(np.arange(ymin, ymax + size, size))

    polygons = []
    for x in cols[:-1]:
        for y in rows[:-1]:
            polygons.append(Polygon([(x,y), (x+size, y), (x+size, y+length), (x, y+size)]))

    grid = gpd.GeoDataFrame({'geometry':polygons})
    grid = grid.set_crs(epsg=4326)
    grid['centroid'] = grid.centroid
    grid = grid.to_crs(epsg=4326)
    return grid

In [114]:
#function that joins IPC data to grid
def ipc_join_to_grid(grid_gdf, polygon_gdf):
    fullgeo = gpd.GeoDataFrame()
    filenum = 1
    print (str(polygon_gdf.shape) + "is the shape of the polygon_gdf")
    for value in polygon_gdf['from'].unique():
        # create a geodataframe for each value
        tempgdf = polygon_gdf[polygon_gdf['from'] == value]
        print (value)
        print (tempgdf.shape)
        month = value[0:3]
        year = value[4:9]
        print (year + "-" + month)
        tempgdf = tempgdf.set_geometry('geometry')
        tempgdf = tempgdf.to_crs(epsg=4326)
        tempgdf = tempgdf.add_suffix("-"+year+"-"+month)
        tempgdf = tempgdf.rename(columns={'geometry-'+year+'-'+month:'geometry'})
        print (year+'-'+month+'-CS')
        # tempgdf = tempgdf.rename(columns={'HA0':year+'-'+month+'-HA0'})
        # the actual spatial join
        grid_gdf = grid_gdf.sjoin(tempgdf, how="left")
        print ("with duplicates dataframe is " + str(grid_gdf.shape))
        grid_gdf = grid_gdf.drop(columns=['index_right'])
        # print column names of grid_gdf
        print (grid_gdf.columns)
        # for duplicate centroid rows we keep the row with the higher CS value
        grid_gdf = grid_gdf.sort_values(by=["overall_phase-"+year+'-'+month], ascending=False)
        grid_gdf = grid_gdf.drop_duplicates(subset=['centroid'], keep='first')
        print ("duplicates removed " + str(grid_gdf.shape))
        filenum += 1
    return grid_gdf

In [ ]:
pk_all = gpd.read_file(get_ipc_data("areas?format=geojson&country=PK&type=A", "txt"))
pk_grid = make_grid(pk_all, 0.1)
pk_grid = ipc_join_to_grid(pk_grid, pk_all)

In [126]:
pk_grid

,geometry,centroid,id-2017-Feb,estimated_population-2017-Feb,period-2017-Feb,from-2017-Feb,to-2017-Feb,color-2017-Feb,anl_id-2017-Feb,title-2017-Feb,...,phase1_color-2021-Oct,phase2_population-2021-Oct,phase2_percent-2021-Oct,phase2_color-2021-Oct,phase3_population-2021-Oct,phase3_percent-2021-Oct,phase3_color-2021-Oct,phase4_population-2021-Oct,phase4_percent-2021-Oct,phase4_color-2021-Oct
2861,"POLYGON ((63.37860 27.29501, 63.47860 27.29501...",POINT (63.42860 27.34501),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,#CDFACD,51527.0,0.30,#FAE61E,42939.0,0.25,#E67800,8588.0,0.05,#C80000
9512,"POLYGON ((69.27860 25.69501, 69.37860 25.69501...",POINT (69.32860 25.74501),12174545,2188239.0,C,Feb 2017,Aug 2017,#C80000,12174517,Sanghar,...,#CDFACD,553390.0,0.35,#FAE61E,316223.0,0.20,#E67800,79056.0,0.05,#C80000
9739,"POLYGON ((69.47860 25.79501, 69.57860 25.79501...",POINT (69.52860 25.84501),12174545,2188239.0,C,Feb 2017,Aug 2017,#C80000,12174517,Sanghar,...,#CDFACD,553390.0,0.35,#FAE61E,316223.0,0.20,#E67800,79056.0,0.05,#C80000
9740,"POLYGON ((69.47860 25.89501, 69.57860 25.89501...",POINT (69.52860 25.94501),12174545,2188239.0,C,Feb 2017,Aug 2017,#C80000,12174517,Sanghar,...,#CDFACD,553390.0,0.35,#FAE61E,316223.0,0.20,#E67800,79056.0,0.05,#C80000
9741,"POLYGON ((69.47860 25.99501, 69.57860 25.99501...",POINT (69.52860 26.04501),12174545,2188239.0,C,Feb 2017,Aug 2017,#C80000,12174517,Sanghar,...,#CDFACD,553390.0,0.35,#FAE61E,316223.0,0.20,#E67800,79056.0,0.05,#C80000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12424,"POLYGON ((71.77860 34.39501, 71.87860 34.39501...",POINT (71.82860 34.44501),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12425,"POLYGON ((71.77860 34.49501, 71.87860 34.49501...",POINT (71.82860 34.54501),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12426,"POLYGON ((71.77860 34.59501, 71.87860 34.59501...",POINT (71.82860 34.64501),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12428,"POLYGON ((71.77860 34.79501, 71.87860 34.79501...",POINT (71.82860 34.84501),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
# print unique pk_all from values
pk_all['to'].unique()

array(['Aug 2017', 'Jul 2019', 'May 2020', 'Jun 2021', 'Mar 2022'],
      dtype=object)

In [131]:
pk_write = pk_grid.drop(columns=['centroid'])

In [133]:
# write pk_grid to geojson file
pk_write.to_file("./synced-data/pk_grid_1.geojson", driver='GeoJSON')

In [127]:
pk_grid = pk_grid.dropna(axis=0, how='all', subset=pk_grid.iloc[:,3:148].columns)

In [ ]:
plot_map(pk_grid, None)

In [120]:
# plots geodataframe on mapbox map with hover.
def plot_map(gdf, interest):
    fig = px.choropleth_mapbox(gdf, geojson=gdf.geometry, locations=gdf.index, color=interest, hover_name=interest, hover_data=[interest], center={"lat": 0, "lon": 0}, mapbox_style="carto-positron", zoom=1, opacity=0.5)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [134]:
pk_reports

[{'id': '12174517',
  'title': 'Pakistan - Acute Food Insecurity May 2017 (Sindh) ',
  'link': 'http://www.ipcinfo.org/ipc-country-analysis/details-map/en/c/1033294/',
  'country': 'PK',
  'year': 2017,
  'condition': 'A',
  'created': '2018-01-17',
  'modified': '2022-02-15'},
 {'id': '13850093',
  'title': 'Pakistan - Acute Food Insecurity April 2019 (Sindh & Balochistan)',
  'link': 'http://www.ipcinfo.org/ipc-country-analysis/details-map/en/c/1152198/?iso3=PAK',
  'country': 'PK',
  'year': 2019,
  'condition': 'A',
  'created': '2019-10-17',
  'modified': '2022-02-15'},
 {'id': '18202000',
  'title': 'Pakistan - Acute Food Insecurity Jan 2020-Khyber Pakhtunkhwa',
  'link': 'http://www.ipcinfo.org/ipc-country-analysis/details-map/en/c/1152728/?iso3=PAK',
  'country': 'PK',
  'year': 2020,
  'condition': 'A',
  'created': '2020-07-06',
  'modified': '2022-02-15'},
 {'id': '23402191',
  'title': 'Pakistan - Acute Food Insecurity March 2021 Sindh+Balochistan',
  'link': 'http://www.ip

In [139]:
# all_pk column names
pk_all.columns

Index(['id', 'estimated_population', 'period', 'from', 'to', 'color', 'anl_id',
       'title', 'overall_phase', 'country', 'year', 'condition',
       'phase3_worse_population', 'phase3_worse_percentage',
       'phase5_population', 'phase5_percent', 'phase5_color',
       'phase1_population', 'phase1_percent', 'phase1_color',
       'phase2_population', 'phase2_percent', 'phase2_color',
       'phase3_population', 'phase3_percent', 'phase3_color',
       'phase4_population', 'phase4_percent', 'phase4_color', 'geometry'],
      dtype='object')

In [141]:
# print unique anl_id values
pk_all['from'].unique()

array(['Feb 2017', 'Oct 2018', 'Nov 2019', 'Mar 2021', 'Oct 2021'],
      dtype=object)